In [1]:
# import 
import numpy as np
import nltk 
import tflearn 
import tensorflow 
from nltk.stem.lancaster import LancasterStemmer
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
stemmer = LancasterStemmer()

Loading the Json file as a dictionary

In [3]:
with open("intents.json") as file:
    data = json.load(file)
data['intents'][:1]

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''}]

Getting the root of the word by tokenizing the word and stemming it 

In [6]:
# try: 
#     # x if you changed the .json uncommented and add a x
#     # if the data is already saved then it runs the save data
#     with open('data.pickle', 'rb') as f:
#         words, labels, training, output = pickle.load(f)
# except:
words = []
labels = []
docs_x = []
docs_y = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        ## seprating the word into a list 
        wrds = nltk.word_tokenize(pattern)
        ## adding all to the list
        words.extend(wrds)
        ## append the tokenize words
        docs_x.append(wrds)
        ## append the tag of the intent
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
## lowering the words to avoid confusing
## removing the ? 
words = [stemmer.stem(w.lower()) for w in words if w not in '?']
## Removing all the duplicates
words = sorted(list(set(words)))

labels = sorted(labels)

## creating a bagged of words in binary to train the model 
## So we can do one hot-encoding with the words
training = []
output = []

## list of tags into one hot-encoding
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]

    for w in words: 
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)
## switching the list into an array for input into a model
training = np.array(training)
output = np.array(output)
## saving the preprocessing 
with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

In [7]:
## defines the input shape for the model
net = tflearn.input_data(shape = [None, len(training[0])])
## adding to the neural network to 2 hidden layers 
## more hidden layers for more complex problem
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
## output layers activation will allow us to get probablity for each neuron.
net = tflearn.fully_connected(net, len(output[0]), activation = 'softmax')
net = tflearn.regression(net)
## Deep Neural Networl model
model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
try:
    model.load('model.tflearn')
except:
    model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)
    model.save('model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\User\Onedrive\Desktop\DSI_GA\chatbot\model.tflearn


In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return np.array(bag)

In [10]:
def chat():
    print('Start talking with the bot! (type q to stop) !')
    while True:
        inp = input('You: ')
        if inp.lower() == 'q':
            break
        ## it will giveout a probablity
        results = model.predict([bag_of_words(inp,words)])[0]
        

        ## gives an index of the largest probablity, so you can display the best answer
        results_index = np.argmax(results)
        tag = labels[results_index]
        if results[results_index] > 0.7:
            print(results)

            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else: 
            print("I didn't really get that, I still need to improve!!!")


In [11]:
wordss = 'gibbersh'

In [12]:
results = model.predict([bag_of_words('gibeersh',words)])[0]
results

array([0.01592351, 0.02787407, 0.19269666, 0.7601327 , 0.0033731 ],
      dtype=float32)

In [ ]:
chat()

Start talking with the bot! (type q to stop) !
